In [1]:
%matplotlib inline

##### Path configuration #####
# Location of Driscoll datasets on your system; Download from 
# https://datadryad.org/stash/dataset/doi:10.5061/dryad.gqnk98sjq
PPC_DATA_PATH = '/home/mer49/Dropbox (Cambridge University)/Datasets/PPC_data/'
# Where to put cache for intermediate results
PY_CACHE_DIR  = './'
# Name of folder in which to cache intermediate processing results
CACHE_NAME    = 'cache'

# Add local libraries to path. A stripped-down copy of neurotools, and 
# some code for extracting the Driscoll datasets, is included. 
# https://github.com/michaelerule/neurotools/blob/master/README.md
import os,sys
sys.path.insert(0,os.path.abspath("./"))
sys.path.insert(0,os.path.abspath("./driscolldatatools/"))

# Pretty ugly: sets up disk caching for procssing Driscoll data
# Caching was useful for exploratory analysis, but is a bit too heavy
# for this paper. I will generate and save the intermediate results, 
# then comment out this code. 
from neurotools.jobs.initialize_system_cache import initialize_caches,cache_test
from neurotools.tools import ensure_dir
ensure_dir(PY_CACHE_DIR+os.sep+CACHE_NAME)
initialize_caches(PY_CACHE_DIR,CACHE_IDENTIFIER=CACHE_NAME)
import neurotools
from   neurotools.nlab import *

# Import Driscoll et al. data wrangling libraries
import ppc_data_loader
ppc_data_loader.path = PPC_DATA_PATH
from ppc_data_loader  import *
from ppc_trial        import *

from config import *

In [62]:
import itertools
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

K = 4

print('Searching for triplets of days with many shared units with separation at least Δ')
for mouse in [1,2,3,4,5]:
    sessions = array(get_session_ids(mouse))
    days     = get_session_days(mouse)
    d        = dict(zip(sessions,days))
    print('mouse %d; days range from %d to %d'%(mouse,min(days),max(days)))
    used = set()
    for Δ in arange(5,40)[::-1]:
        subsets  = [sorted(s) for s in findsubsets(sessions,K)]
        subsets  = [tuple(s) for s in subsets if 
                    all([d[s[i+1]]>d[s[i]]+Δ for i in range(K-1)])]
        if len(subsets)>0:
            units = [get_units_in_common(mouse,s)[0] for s in progress_bar(subsets)]
            best  = argmax(amap(len,units))
            if not subsets[best] in used: 
                dstr  = ', '.join([str(d[s]) for s in subsets[best]])
                print('  Δ=%02d\t'%Δ,subsets[best],
                      '\t(days %s): \t%d tracked units'%(dstr,len(units[best])))
                used.add(subsets[best])

Searching for triplets of days with many shared units with separation at least Δ
mouse 1; days range from 1 to 59
  Δ=15	 (1, 15, 29, 52) 	(days 1, 17, 34, 59): 	19 tracked units      
  Δ=14	 (1, 15, 28, 41) 	(days 1, 17, 33, 48): 	20 tracked units      
  Δ=13	 (1, 14, 28, 41) 	(days 1, 15, 33, 48): 	26 tracked units      
  Δ=12	 (1, 14, 28, 39) 	(days 1, 15, 33, 46): 	31 tracked units      
  Δ=09	 (1, 10, 28, 39) 	(days 1, 11, 33, 46): 	33 tracked units      
  Δ=06	 (2, 15, 20, 39) 	(days 2, 17, 24, 46): 	44 tracked units      
mouse 2; days range from 1 to 21                                      
mouse 3; days range from 1 to 49
  Δ=14	 (1, 14, 25, 42) 	(days 1, 16, 31, 49): 	52 tracked units      
  Δ=12	 (1, 12, 21, 35) 	(days 1, 14, 27, 42): 	70 tracked units      
  Δ=11	 (1, 14, 22, 35) 	(days 1, 16, 28, 42): 	74 tracked units      
  Δ=10	 (1, 14, 25, 35) 	(days 1, 16, 31, 42): 	89 tracked units      
  Δ=09	 (1, 10, 18, 30) 	(days 1, 12, 22, 36): 	90 tracked units      
 